# How many permission requests and what kind of permission will a company ask for?

 possible important variables:  country and google rank

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

import urllib.request as urllib2 # For python3
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm

In [2]:
gdpr = pd.read_csv("GDPRDataAge5.csv")

In [3]:
gdpr.head()

,package_name,country,category,google_rank,pregdpr.permissions,permission_count,Unnamed: 6,package_name.1,country.1,category.1,google_rank.1,postgdpr.permissions,permission_count.1
0,bluphant.games.gravity,United States,Ages 5 & Under,358,[],0,NaN,com.amazon.tahoe,Germany,Ages 5 & Under,176,"['android.permission.READ_PHONE_STATE', 'andro...",67
1,air.com.pepiplay.pepibathliteandroid,United States,Ages 5 & Under,383,['android.permission.INTERNET'],1,NaN,com.amazon.tahoe,United States,Ages 5 & Under,49,"['android.permission.READ_PHONE_STATE', 'andro...",66
2,com.intellijoy.sightwords.lite,United States,Ages 5 & Under,466,['android.permission.INTERNET'],1,NaN,bluepin_app.cont.glbburi,South Korea,Ages 5 & Under,357,"['android.permission.CAMERA', 'android.permiss...",33
3,com.kevinbradford.games.barnyardgamesfree,United States,Ages 5 & Under,195,['com.android.vending.BILLING'],1,NaN,bluepin_app.cont.his_show_kor,South Korea,Ages 5 & Under,302,"['android.permission.INTERNET', 'android.permi...",33
4,com.kevinbradford.games.firstgrade,United States,Ages 5 & Under,170,['com.android.vending.BILLING'],1,NaN,bluepin_app.cont.ikooo_kor,South Korea,Ages 5 & Under,86,"['android.permission.INTERNET', 'android.permi...",33


In [4]:
gdpr.drop(['Unnamed: 6'], axis = 1, inplace = True)


In [5]:
#checking for null values
gdpr.isnull().sum()

package_name            0
country                 0
category                0
google_rank             0
pregdpr.permissions     0
permission_count        0
package_name.1          0
country.1               0
category.1              0
google_rank.1           0
postgdpr.permissions    0
permission_count.1      0
dtype: int64

In [6]:
gdpr.columns

Index(['package_name', 'country', 'category', 'google_rank',
       'pregdpr.permissions', 'permission_count', 'package_name.1',
       'country.1', 'category.1', 'google_rank.1', 'postgdpr.permissions',
       'permission_count.1'],
      dtype='object')

In [7]:
gdpr_pre = gdpr[["package_name", 'country', 'category', 'google_rank',
       'pregdpr.permissions', 'permission_count']]

gdpr_post = gdpr[['package_name.1',
       'country.1', 'category.1', 'google_rank.1', 'postgdpr.permissions',
       'permission_count.1']]

In [8]:
gdpr_pre.shape

(902, 6)

In [9]:
gdpr_post.shape

(902, 6)

In [10]:
gdpr_post[gdpr_post['package_name.1'] == "air.com.pepiplay.pepibathliteandroid"]

,package_name.1,country.1,category.1,google_rank.1,postgdpr.permissions,permission_count.1
873,air.com.pepiplay.pepibathliteandroid,United States,Ages 5 & Under,504,['android.permission.INTERNET'],1


In [11]:
gdpr_post = gdpr_post.rename(columns={"package_name.1": "package_name", "country.1": 'country'})

gdpr_merge = gdpr_pre.merge(gdpr_post, on = ['package_name','country'], how = 'inner')



In [12]:
gdpr_merge.shape

(902, 10)

In [24]:
unique_permissions = []


# we can start by creating a list of each row's permissionsall_permissions (which will be lists too)
permissions = [i for i in gdpr_merge['pregdpr.permissions']]

for i in permissions:
    new_permissions = i.strip('][').split(', ')
   
    for x in new_permissions:
    # check if exists in unique_permissions list or not
        if x not in unique_permissions:
            unique_permissions.append(x)

print('There are', len(unique_permissions), 'unique permissions in the dataset.','\n')
for i in unique_permissions:
    print(i.strip('\''))

There are 338 unique permissions in the dataset. 


android.permission.INTERNET
com.android.vending.BILLING
android.permission.VIBRATE
android.permission.WRITE_EXTERNAL_STORAGE
android.permission.ACCESS_NETWORK_STATE
android.permission.ACCESS_WIFI_STATE
android.permission.READ_PHONE_STATE
android.permission.RECORD_AUDIO
android.permission.CAMERA
android.permission.DISABLE_KEYGUARD
android.permission.WAKE_LOCK
com.android.vending.CHECK_LICENSE
android.permission.READ_EXTERNAL_STORAGE
com.sec.android.iap.permission.BILLING
android.permission.MODIFY_AUDIO_SETTINGS
android.permission.CHANGE_WIFI_STATE
com.google.android.c2dm.permission.RECEIVE
com.funcov.babyplay.permission.C2D_MESSAGE
android.permission.RECEIVE_BOOT_COMPLETED
android.permission.ACCESS_COARSE_LOCATION
android.permission.ACCESS_FINE_LOCATION
forqan.tech.families3.permission.C2D_MESSAGE
com.iabuzz.MemoryMatchFood.permission.C2D_MESSAGE
com.juniorpear.animal_sound.permission.C2D_MESSAGE
forqan.tech.families2.permission.C2D_ME

In [14]:
gdpr_merge.corr()

,google_rank,permission_count,google_rank.1,permission_count.1
google_rank,1.000000,-0.105466,0.642703,-0.063975
permission_count,-0.105466,1.000000,-0.129700,0.695174
google_rank.1,0.642703,-0.129700,1.000000,-0.054363
permission_count.1,-0.063975,0.695174,-0.054363,1.000000


# Testing Out Some Models